In [79]:
import numpy as np
import os, glob, re
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
py.init_notebook_mode(connected=True)

In [9]:
def list_files(fpattern):
    files = glob.glob(fpattern)
    grab_digit = lambda f : int(re.search("([0-9]*)\.[a-zA-Z]*$",f).groups()[-1])
    files.sort(key=lambda f: grab_digit(f) )
    return files

In [72]:
training_dir = "/Users/afq/Google Drive/networks/"
logdir = training_dir+"training_water_slgc_logp"
archs = os.listdir(logdir)
archs.sort()

In [73]:
archs

['Classifying_1,1,12,24,sigmoid',
 'Classifying_1,1,6,12,sigmoid',
 'Classifying_1,5,6,12,sigmoid',
 'Classifying_2,1,12,24,sigmoid',
 'Classifying_2,1,6,12,sigmoid',
 'Classifying_2,5,12,24,sigmoid',
 'Deep_1,(),1,(12,12,12)',
 'Deep_1,(12),1,(12,12,12)',
 'Poly_2,7']

In [74]:
import itertools
num = len(archs)
offs = range(int(np.ceil(len(archs)**0.5)))
offsets = itertools.product(offs,offs)
offsets=list(offsets)

In [95]:
from scipy.spatial import Delaunay
def make_plot(D, offset=(0,0), c='gray', name='', **kwargs):
    tri = Delaunay(D[:,4:6])
    simplices = tri.simplices
    return plotly_trisurf(x=D[:,0]+1.5*offset[0],y=D[:,1],z=D[:,2]+1.5*offset[1],
                         simplices=simplices)

In [86]:
def make_plot(D, offset=(0,0), c='gray', name='', **kwargs):
    return go.Scatter3d(x=D[:,0]+1.5*offset[0],y=D[:,1],z=D[:,2]+1.5*offset[1],
                mode='markers',marker=dict(size=2,color=D[:,-1]),name=name,**kwargs)

In [ ]:
def colorify(z,colormap):
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean]

In [137]:
from scipy.spatial import Delaunay
def make_plot(D, offset=(0,0),colormap=cm.RdBu,  c='gray', name='', **kwargs):
    tri = Delaunay(D[:,4:6])
    simplices = tri.simplices
#     min_c = 
#     facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in D[:,-1]]
    return go.Mesh3d(x=D[:,0]+1.5*offset[0],y=D[:,1],z=D[:,2]+1.5*offset[1],
                         i=simplices[:,0],j=simplices[:,1],k=simplices[:,2],
#                      vertexcolor=D[:,-1],
                     intensity=D[:,-1]/np.max(D[:,-1]),
                     name=name,showscale = False,**kwargs)

In [118]:
import matplotlib.cm as cm

def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value zval to a corresponding color in the colormap

    if vmin>vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    R, G, B, alpha=colormap(t)
    return 'rgb('+'{:d}'.format(int(R*255+0.5))+','+'{:d}'.format(int(G*255+0.5))+\
           ','+'{:d}'.format(int(B*255+0.5))+')'
def tri_indices(simplices):
    #simplices is a numpy array defining the simplices of the triangularization
    #returns the lists of indices i, j, k

    return ([triplet[c] for triplet in simplices] for c in range(3))

def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, plot_edges=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangularization;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data

    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)# vertices of the surface triangles     
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean]
    I,J,K=tri_indices(simplices)

    triangles=go.Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor,
                     i=I,
                     j=J,
                     k=K,
                     name=''
                    )

    if plot_edges is None:# the triangle sides are not plotted 
        return [triangles]
    else:
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None]   for T in tri_vertices]  for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]

        #define the lines to be plotted
        lines=go.Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=dict(color= 'rgb(50,50,50)', width=1.5)
               )
        return [triangles, lines]

In [115]:
surfaces = []
for arch in archs:
    files = list_files(logdir+'/'+arch+'/surf_*.csv')
    try:
        dat = np.loadtxt(files[-1],delimiter=",",skiprows=1)
    except:
        dat = np.zeros((1,3))
    surfaces.append( dat )

In [139]:
plot_data = [make_plot(s,o,name=n) for s,o,n in zip(surfaces,offsets,archs)]
layout = go.Layout(showlegend=True)
fig = go.Figure(data=plot_data,layout=layout)
py.plot(fig,filename='networks.html')

'file:///Users/afq/Documents/Dropbox/ML/primaryautoencoder/networks.html'